<a href="https://colab.research.google.com/github/ShashankKamath/DecisionTree_ML/blob/master/DT_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

In [2]:
from google.colab import files
uploaded = files.upload()


Saving hw2_question1.csv to hw2_question1.csv


In [0]:
import pandas as pd
import numpy as np
data = pd.read_csv("hw2_question1.csv")
train_set=data.values.tolist()
header=["Thickness", "Cellsize", "Cellshape", "Adhesion","Ecellsize","Nuclei","Chromatin","Nucleoli","Mitosis","Class"]


In [22]:
#class_counts=data['Class'].value_counts()
#print('Class Benign Count: ' + repr(class_counts[2]))
#print('Class Malignant Count: ' + repr(class_counts[4]))

[5, 1, 1, 1, 2, 1, 3, 1, 1, 2]

In [7]:
data.head()

,Thickness,Cellsize,Cellshape,Adhesion,Ecellsize,Nuclei,Chromatin,Nucleoli,Mitosis,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [24]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])
unique_vals(train_set, 1)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [25]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
  
class_counts(train_set)

{2: 444, 4: 239}

In [26]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)
is_numeric(7)

True

In [0]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))


In [57]:
q = Question(6,7)
example = train_set[0]
q.match(example)
q

Is Chromatin >= 7?

In [0]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [0]:
true_rows, false_rows = partition(train_set, Question(0, 2))

This will give all values less than two in the first column

In [0]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity 

In [0]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [49]:
current_uncertainty = gini(train_set)
current_uncertainty

0.45495606541633354

In [0]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [64]:
best_gain, best_question = find_best_split(train_set)
best_question

Is Cellsize >= 3?

In [0]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [0]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [0]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [0]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [0]:
my_tree = build_tree(train_set)

In [70]:
print_tree(my_tree)

Is Cellsize >= 3?
--> True:
  Is Cellshape >= 3?
  --> True:
    Is Cellsize >= 5?
    --> True:
      Is Adhesion >= 2?
      --> True:
        Is Nucleoli >= 3?
        --> True:
          Predict {4: 136}
        --> False:
          Is Nucleoli >= 2?
          --> True:
            Is Chromatin >= 7?
            --> True:
              Predict {4: 4}
            --> False:
              Predict {2: 1}
          --> False:
            Predict {4: 22}
      --> False:
        Is Thickness >= 8?
        --> True:
          Predict {4: 7}
        --> False:
          Is Mitosis >= 3?
          --> True:
            Predict {4: 2}
          --> False:
            Predict {2: 2}
    --> False:
      Is Nuclei >= 3?
      --> True:
        Is Thickness >= 7?
        --> True:
          Is Nuclei >= 8?
          --> True:
            Predict {4: 22}
          --> False:
            Is Adhesion >= 5?
            --> True:
              Is Mitosis >= 2?
              --> True:
              

Predicting the Class

In [0]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [73]:
classify(train_set[0], my_tree)

{2: 396}

In [0]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [76]:
print_leaf(classify(train_set[], my_tree))

{4: '100%'}

In [80]:
test_set=train_set[]
for row in test_set:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Pre